# Day-4 Decision Trees

Decision Trees are very strong Machine Learning Algorith and are fundamental part of Random Forests

## Classification

### Training and Visualizing a Decision Tree

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [2]:
iris = load_iris()

X = iris['data'][:, 2:]  # petal length and width
y = iris['target']

In [3]:
tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [4]:
from sklearn.tree import export_graphviz

In [5]:
export_graphviz(tree_clf, out_file='iris_tree.dot', feature_names=iris.feature_names[2:], class_names=iris.target_names, rounded=True, filled=True)

In [6]:
!dot -Tpng iris_tree.dot -o iris_tree.png

zsh:1: command not found: dot


![iris tree](iris_tree.png)

### Making Predictions

- **Root Node** => Top Most Node (depth = 0)
- **Child Nodes** => All Nodes below Root Node with Childs
- **Leaf Node**  => Node that does not have any Child Nodes

- **Samples** => how many training instances does the split apply to
- **Value** => Tell about how many training instances of each class
- **Gini** => Measures the Impurity

Gini = 0 ('Pure') all training instances it applies belongs to the same class

**Gi = 1 - Summation_k(pik^2)**

where pik is the ratio of class k instances amoung the training instances of the ith node

Example for depth 2 and class veriscolor - Gini = 1 - (0/54)^2 - (49/54)^2 - (5/54)^2 ~ 0.168

**Note**- Sklearn uses CART algorith which only produces Binary Trees (yes/no). However other algorithms like ID3 can produce Decision Trees which have nodes with more than two child nodes.

Decision Trees can also return probabilities. To do so they first traverse through Tree to reach the leaf node and then return the ratio of trainig instaces of the class k (like 49/54 for Versicolor)

### CART Training Algorithm

**Classification and Regression Tree Algorithm** works by splitting training set into subsets using single feature k ans a threshold tk (like petal length <= 2.45). It searches for k and tk that produce purest subsets. It tries to **mininmise**

**J(k, tk) = (m_left/m)\*G_left + (m_right/m)\*G_right**  (Cost Function)

where G is the ginni value (impurity) and m is the number of training examples in left/right subset

max_depth, min_samples_split, min_samples_leaf, min_weight_fraction_left and max_leaf_nodes can be used to control the depth of the tree.

#### Computational Complexity

Finding the **Optimal Solution** requires O(exp(m)) time which makes it very difficult even for small datasets thus we have to settle with **Reasonable Good** solution.

Making Prediction requires going through O(log2(m)) nodes. Since each node requires checking value of one feature the over complexity is O(log2(m)) which is indipendent of number of features.

Training Complexity is about O(n x m log2(m))

#### Gini vs Entropy

Entropy can also be used instead of Gini

**Entropy = - Summation_k(pik\*log2(pik))** where, pik != 0

Usually both produce same results, Gini is a bit fast to compute. Gini isolates most frequent classes into there own brach of trees whereas entropy tends to produce overall more balanced trees.

#### Regularization

**max_depth**, min_samples_split (min number of samples a node must have before splitting), min_samples_leaf (min number of samples a leaf node must have before splitting), min_weight_fraction_leaf (same as min_samples_leaf just the ratio is used), max_leaf_nodes (max number of leaf nodes) and max_features can be used to regularize the decision tree. Incresing the min_ hyperparameter or decreasing max_ hyperparameter.

Many algorithms first train decision trees without restrictions, then pruning (deleting) unnecessary nodes. This can be done by using chi square tests etc.

In [7]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [8]:
tree_clf.predict([[5, 1.5]])

array([1])

### Regression

In [9]:
from sklearn.tree import DecisionTreeRegressor

In [10]:
tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [11]:
export_graphviz(tree_reg, out_file='iris_tree_reg.dot', feature_names=iris.feature_names[2:], class_names=iris.target_names, rounded=True, filled=True)

In [12]:
!dot -Tpng iris_tree_reg.dot -o iris_tree_reg.png

zsh:1: command not found: dot


![tree reg](iris_tree_reg.png)

- #### Instead of predicting a class it predicts a value, which is the average target value of all training examples in that node.
- #### Instead of Gini Function it uses MSE


**J(k, tk) = (m_left/m)\*MSE_left + (m_right/m)\*MSE_right**  (Cost Function)

where, **MSE_node = Summation_i(y_node - y_i)^2** and **y_node = (1/m_node)\*Summation_i(y_i)**

#### Limitations of Decision Trees

- They have orthogonal boundrie which makes them sensitive to training set rotation. To help with this techniques like PCA can be used.
- Very Sensitive to small variations in training data

### Iterative Dichotomiser 3 (ID3)

The algorithm creates a multiway tree, finding for each node (i.e. in a greedy manner) the categorical feature that will yield the largest information gain for categorical targets. Trees are grown to their maximum size and then a pruning step is usually applied to improve the ability of the tree to generalise to unseen data.

### C4.5

C4.5 is similar to ID3, but removed the restriction that features must be categorical by dynamically defining a discrete attribute (based on numerical variables) that partitions the continuous attribute value into a discrete set of intervals. C4.5 converts the trained trees (i.e. the output of the ID3 algorithm) into sets of if-then rules. This accuracy of each rule is then evaluated to determine the order in which they should be applied. Pruning is done by removing a rule’s precondition if the accuracy of the rule improves without it.

### C5

C5.0 is under a proprietary license. It uses less memory and builds smaller rulesets than C4.5 while being more accurate.

### Chi_Square Automatic Interaction Detection (CHAID)

To find the most dominant feature, chi-square tests will use that is also called CHAID whereas ID3 uses information gain, C4.5 uses gain ratio and CART uses the GINI index.

The formula of chi-square:-

√((y – y’)2 / y’)

where y is actual and y’ is expected.
